In [15]:
RK = os.environ.get('RAPID_KEY')

!{sys.executable} -m pip install requests pandas numpy scipy plotly chart_studio
import requests

Defaulting to user installation because normal site-packages is not writeable


In [17]:
payload = '''{"urls": ["https://github.com/novuhq/novu"]}'''
headers = {'X-RapidApi-Key': RK, 'X-RapidApi-Host': 'lowendinsight2.p.rapidapi.com', 'Content-Type': 'application/json'}
r = requests.post('https://lowendinsight2.p.rapidapi.com/analyze/', data=payload, headers=headers)

import json
import time

j = json.loads(r.content)
print(j)
while j['state'] == "incomplete":
  r = requests.get("https://lowendinsight2.p.rapidapi.com/analyze/{}".format(j["uuid"]))
  print(r)
  j = json.loads(r.content)
  time.sleep(2)
  print("not complete yet, sleeping...")
    

{'uuid': 'd442754a-9676-11ed-b059-06aa01f57570', 'state': 'complete', 'report': {'repos': [{'header': {'uuid': 'b94e63f2-9676-11ed-aa4a-5641c3ad0f84', 'start_time': '2023-01-17T14:53:19.642069Z', 'source_client': 'lei_worker', 'repo': 'https://github.com/novuhq/novu', 'library_version': '0.8.1', 'end_time': '2023-01-17T14:53:37.750763Z', 'duration': 18}, 'data': {'risk': 'medium', 'results': {'top10_contributors': [{'name': 'Dima Grossman', 'merges': 1035, 'last_contribution_date': '2023-01-17T16:07:48+02:00', 'email': 'dima@grossman.io', 'contributions': 2233}, {'name': 'Gosha', 'merges': 126, 'last_contribution_date': '2023-01-10T22:38:23+02:00', 'email': 'djabarov.george@gmail.com', 'contributions': 1053}, {'name': 'ainouzgali', 'merges': 398, 'last_contribution_date': '2023-01-13T11:10:19+02:00', 'email': 'ainouzgali@gmail.com', 'contributions': 894}, {'name': 'David Söderberg', 'merges': 84, 'last_contribution_date': '2023-01-16T09:24:26+01:00', 'email': 'granskog_@hotmail.com', '

In [46]:


results = j["report"]["repos"][0]["data"]["results"]
#print(results)
total_commits = j["report"]["repos"][0]["data"]["git"]["total_commits_on_default_branch"]
top10 = results["top10_contributors"]
#print(top10)

for c in top10:
    #print("{} [{}]: {} {}".format(c["name"], c["email"], c["contributions"], round((c["contributions"]/total_commits)*100,2)))
    c.update({'contribution_percentage': round((c["contributions"]/total_commits)*100,2)})

import pandas as pd
import numpy as np
import scipy as sp
import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.figure_factory as ff

df = pd.DataFrame.from_dict(top10)
df = df.drop('email', axis=1)
table = ff.create_table(df)

#plotly.offline.plot(table, filename='lei-table1.png')
table.update_layout(width=1024)
table.show()

KeyError: "['email'] not found in axis"

In [ ]:
import plotly.express as px
fig = px.bar(x=df['name'].to_list(), y=df['contributions'].to_numpy())
fig.show()